In [74]:
#imports
import json
import requests
import os

In [75]:
API_KEY = os.getenv("API_KEY")
API_HOST = os.getenv("API_HOST")
category_id = "283155"

In [77]:
API_KEY = "8c54ca9e46msh8eefe6b139cd3b9p1a8b28jsna71252a67483"
API_HOST = "real-time-amazon-data.p.rapidapi.com"

# EXTRACTION

In [79]:
def extract_api_data():
    #getting data of amazon data from rapidapi
    url = "https://real-time-amazon-data.p.rapidapi.com/products-by-category"

    #setting key and host in headers
    headers = {
        "X-RapidAPI-Key": API_KEY, #defined in .env file
        "X-RapidAPI-Host": API_HOST #defined in .env file
    }

    #getting the category books
    params = {"category_id": category_id}

    response = requests.get(url, headers=headers, params=params)
    json_data = response.json()

    #checking if the response is successful
    if response.status_code == 200:
        print("Data extracted successfully!")
    else:
        print(f"Failed to extract data. Status code: {response.status_code}")
    return json_data


# TRANSFORMATION

In [80]:
import pandas as pd

In [81]:
json_data = extract_api_data()

Data extracted successfully!


In [82]:
data = json_data["data"]["products"]

In [84]:
books_df = pd.DataFrame(data)
books = books_df.rename(columns={"asin": "product_id"}) #renaming column asin to product_id
books["product_price"] = (books["product_price"]
                          .str.replace("$", "", regex=False) #removing $ sign
                          .str.replace(",", ".", regex=False) #replacing , with . if exist
                          .astype(float)) #converting to float
books["product_original_price"] = (books["product_original_price"] #same this for original price
                                    .str.replace("$", "", regex=False)
                                    .str.replace(",", ".", regex=False)
                                    .astype(float))
books["discount_percentage"] = round((books["product_original_price"] - books["product_price"]) / books["product_original_price"] * 100, 2) #creating discount percentage column
books["product_byline"] = (books["product_byline"]
                           .str.replace("by ", "", regex=False)) #removing 'by ' from product_byline column
books = books.drop(columns=["book_formats"]) #dropping book_formats column (duplicate info)
books["product_byline_links"] = books["product_byline_links"].apply(lambda x: x[0]["link"] if isinstance(x, list) and len(x) > 0 else None) #extracting only the  byline link from product_byline_links list
books.head()

,product_id,product_title,product_price,product_original_price,currency,product_star_rating,product_num_ratings,book_format,product_url,product_photo,...,is_best_seller,is_amazon_choice,is_prime,climate_pledge_friendly,sales_volume,delivery,has_variations,product_byline,product_byline_links,discount_percentage
0,B0DQCYYD94,What She Saw,1.99,5.99,USD,4.4,6301,Kindle,https://www.amazon.com/dp/B0DQCYYD94,https://m.media-amazon.com/images/I/81Or6gWEIX...,...,False,False,False,False,None,None,False,"Mary Burton | Nov 1, 2025",https://www.amazon.com/Mary-Burton/e/B001IYV45...,66.78
1,B0DZY65K1X,The Restoration Garden: A Novel,1.99,4.99,USD,4.5,4808,Kindle,https://www.amazon.com/dp/B0DZY65K1X,https://m.media-amazon.com/images/I/91RwHjXkO6...,...,False,False,False,False,None,None,False,"Sara Blaydes | Nov 1, 2025",https://www.amazon.com/Sara-Blaydes/e/B0CVY12R...,60.12
2,B0DWXY3TNK,The Fix: A Novel,1.99,4.99,USD,4.5,3281,Kindle,https://www.amazon.com/dp/B0DWXY3TNK,https://m.media-amazon.com/images/I/81cHT2yjjX...,...,False,False,False,False,None,None,False,"Mia Sheridan | Nov 1, 2025",https://www.amazon.com/Mia-Sheridan/e/B00C32SV...,60.12
3,B07PCV9DSZ,Audible Membership,0.00,NaN,USD,None,0,Audible Audiobook,https://www.amazon.com/dp/B07PCV9DSZ,https://m.media-amazon.com/images/I/21MjMFj7JA...,...,False,False,False,False,None,None,False,NaN,None,NaN
4,B0F19G66SF,The Moorwitch,1.99,4.99,USD,4.5,1581,Kindle,https://www.amazon.com/dp/B0F19G66SF,https://m.media-amazon.com/images/I/81yX8qj46j...,...,False,False,False,False,None,None,False,"Jessica Khoury | Nov 1, 2025",https://www.amazon.com/Jessica-Khoury/e/B00A42...,60.12


In [85]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_id                   16 non-null     object 
 1   product_title                16 non-null     object 
 2   product_price                16 non-null     float64
 3   product_original_price       15 non-null     float64
 4   currency                     16 non-null     object 
 5   product_star_rating          15 non-null     object 
 6   product_num_ratings          16 non-null     int64  
 7   book_format                  16 non-null     object 
 8   product_url                  16 non-null     object 
 9   product_photo                16 non-null     object 
 10  product_num_offers           16 non-null     int64  
 11  product_minimum_offer_price  16 non-null     object 
 12  is_best_seller               16 non-null     bool   
 13  is_amazon_choice      

# LOAD

In [92]:
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_PORT = os.getenv("MYSQL_PORT")
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
MYSQL_DB = os.getenv("MYSQL_DB")

In [ ]:
pip install mysql-connector-python
pip install mysql-connector-python pandas

import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

def load_to_mysql(df):
    host=MYSQL_HOST
    user=MYSQL_USER,
    password=MYSQL_PASSWORD
    database = "amazon_data_db"

    engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

    df.to_sql(name="amazon_books", con=engine, if_exists="append", index=False)

    print("Data inserted successfully!")

load_to_mysql(books)